In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical

In [2]:
data_dir = './Image_data/'
interior_types = ['Bohemian', 'Coastal', 'Industrial', 'Scandinavian']

In [3]:
def preprocess_images(data_dir, interior_types, image_size):
    X_data = []
    y_data = []
    
    for interior_type_id, interior_type in enumerate(interior_types):
        interior_type_path = os.path.join(data_dir, interior_type)
        for img_name in os.listdir(interior_type_path):
            img_path = os.path.join(interior_type_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, image_size)
            img = img.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]
            X_data.append(img)
            y_data.append(interior_type_id)
    
    X_data = np.array(X_data)
    y_data = np.array(y_data)
    y_data = to_categorical(y_data, num_classes=len(interior_types))  # One-hot encode the labels
    
    return X_data, y_data


In [4]:
image_size = (224, 224)

In [5]:
X_data, y_data = preprocess_images(data_dir, interior_types, image_size)

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(interior_types), activation='softmax'))

In [9]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
batch_size = 32
epochs = 10

In [11]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/10
45/45 [==============================] - 89s 2s/step - loss: 1.5348 - accuracy: 0.3756 - val_loss: 1.0112 - val_accuracy: 0.5042
Epoch 2/10
45/45 [==============================] - 80s 2s/step - loss: 0.9748 - accuracy: 0.5567 - val_loss: 0.8448 - val_accuracy: 0.6225
Epoch 3/10
45/45 [==============================] - 80s 2s/step - loss: 0.9498 - accuracy: 0.5814 - val_loss: 0.9399 - val_accuracy: 0.5606
Epoch 4/10
45/45 [==============================] - 83s 2s/step - loss: 0.8030 - accuracy: 0.6540 - val_loss: 0.7421 - val_accuracy: 0.6901
Epoch 5/10
45/45 [==============================] - 80s 2s/step - loss: 0.6973 - accuracy: 0.7047 - val_loss: 0.7881 - val_accuracy: 0.6648
Epoch 6/10
45/45 [==============================] - 80s 2s/step - loss: 0.5962 - accuracy: 0.7632 - val_loss: 0.7541 - val_accuracy: 0.6676
Epoch 7/10
45/45 [==============================] - 80s 2s/step - loss: 0.4738 - accuracy: 0.8168 - val_loss: 0.7755 - val_accuracy: 0.6732
Epoch 8/10
45/45 [==

In [ ]:
model.save('interior_type_model.h5')